In [4]:
!pip install -U langchain-ollama


  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install -U langchain-core



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install -U langchain langchain-community langchain-ollama


   ---------------------------------------- 1.0/1.0 MB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 2.5/2.5 MB 4.3 MB/s eta 0:00:00
   ---------------------------------------- 12.9/12.9 MB 13.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.2
    Uninstalling langchain-text-splitters-0.3.2:
      Successfully uninstalled langchain-text-splitters-0.3.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.5
    Uninstalling langchain-community-0.3.5:
      Successfully uninstalled langchain-community-0.3.5


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.33.0 requires numpy<2.0.0,>=1.17, but you have numpy 2.2.6 which is incompatible.
evidently 0.4.0 requires pydantic<2,>=1.10, but you have pydantic 2.9.2 which is incompatible.
gradio 3.40.1 requires numpy~=1.0, but you have numpy 2.2.6 which is incompatible.
numba 0.57.1 requires numpy<1.25,>=1.21, but you have numpy 2.2.6 which is incompatible.
scipy 1.9.2 requires numpy<1.26.0,>=1.18.5, but you have numpy 2.2.6 which is incompatible.
spacy 3.4.3 requires pydantic!=1.8,!=1.8.1,<1.11.0,>=1.7.4, but you have pydantic 2.9.2 which is incompatible.
spacy 3.4.3 requires typer<0.8.0,>=0.3.0, but you have typer 0.9.0 which is incompatible.
streamlit 1.25.0 requires numpy<2,>=1.19.3, but you have numpy 2.2.6 which is incompatible.
streamlit 1.25.0 requires tenacity<9,>=8.

In [1]:
!ollama pull llama3

pulling manifest â ‹ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â � pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest 
pulling 6a0746a1ec1a... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 4.7 GB                         
pulling 4fa551d4f938... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  12 KB                         
pulling 8ab4849b038c... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  254 B                         
pulling 577073ffcc6c... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  110 B                         
pulling 3f8eb4da87fa... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [12]:
import csv
import pymongo
import pandas as pd
import json
from langchain.llms import Ollama  # Correct import
from datetime import datetime
#from langchain_ollama.chat_models import ChatOllama


# --- Configuration ---
MONGO_URI = "mongodb://127.0.0.1:27017/"
DB_NAME = "Automated_query"
COLLECTION_NAME = "products_data"
OLLAMA_MODEL = "llama3"  # Replace with your preferred Ollama model
QUERIES_LOG_FILE = "Queries_generated.txt"

# --- Load CSV Data to MongoDB ---
def load_csv_to_mongodb(csv_file_path, mongo_host, database_name, collection_name):
    """Loads data from a CSV file into a MongoDB collection."""
    try:
        client = pymongo.MongoClient(mongo_host)
        db = client[database_name]
        collection = db[collection_name]
        collection.delete_many({})  # Clear existing data
        df = pd.read_csv(csv_file_path)
        data = df.to_dict(orient='records')
        if data:
            collection.insert_many(data)
            print(f"Successfully loaded {len(data)} records from '{csv_file_path}' "
                  f"into '{collection_name}' in '{database_name}'.")
        else:
            print(f"The CSV file '{csv_file_path}' is empty.")
    except Exception as e:
        print(f"Error loading CSV to MongoDB: {e}")
    finally:
        if 'client' in locals():
            client.close()

# --- Generate MongoDB Query using LLM (Ollama) ---
#from langchain_ollama import OllamaLLM  # updated import

import re

def generate_mongodb_query(user_input, collection_fields):
    """Generates a MongoDB query string using Ollama."""
    try:
        llm = Ollama(base_url='http://localhost:11434', model=OLLAMA_MODEL)
        prompt_template = f"""
        You are a helpful assistant that translates user queries into MongoDB find() queries.
        The available fields in the MongoDB collection are: {', '.join(collection_fields)}.

        User Query: {{user_query}}
        MongoDB Query (as a Python dictionary):
        """
        prompt = prompt_template.format(user_query=user_input)
        raw_response = llm.invoke(prompt).strip()

        # Extract content between triple backticks or first JSON-like structure
        match = re.search(r"```(?:json)?\s*([\s\S]+?)```", raw_response)
        if match:
            cleaned = match.group(1)
        else:
            # Fallback: try to find the first { } block
            match = re.search(r"(\{[\s\S]+\})", raw_response)
            cleaned = match.group(1) if match else raw_response

        return cleaned
    except Exception as e:
        print(f"Error generating query with LLM: {e}")
        return None



# --- Retrieve and Present Data ---
def retrieve_and_present_data(mongo_host, database_name, collection_name, mongodb_query_string, output_format="display", output_filename=None):
    """Retrieves data from MongoDB using the generated query and presents it."""
    try:
        client = pymongo.MongoClient(mongo_host)
        db = client[database_name]
        collection = db[collection_name]

        try:
            query = json.loads(mongodb_query_string)
        except json.JSONDecodeError:
            print("Error: Generated MongoDB query is not valid JSON.")
            return None

        results = list(collection.find(query))

        if results:
            print("\nRetrieved Data:")
            df = pd.DataFrame(results)
            if "_id" in df.columns:
                df = df.drop(columns=["_id"])

            if output_format == "display":
                print(df.to_string())
                return df
            elif output_format == "save" and output_filename:
                df.to_csv(output_filename, index=False)
                print(f"Data saved to '{output_filename}'")
                return df
            else:
                print("Invalid output format or filename provided.")
                return None
        else:
            print("No data found matching the query.")
            return pd.DataFrame()

    except Exception as e:
        print(f"Error retrieving data from MongoDB: {e}")
        return None
    finally:
        if 'client' in locals():
            client.close()

# --- Main Interaction Function ---
def main():
    csv_file = "sample_data.csv"
    load_csv_to_mongodb(csv_file, MONGO_URI, DB_NAME, COLLECTION_NAME)

    client = pymongo.MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    first_document = collection.find_one()

    if first_document:
        fields = list(first_document.keys())
        if "_id" in fields:
            fields.remove("_id")

        while True:
            user_question = input("\nEnter your data query (or type 'exit' to quit): ")
            if user_question.lower() == 'exit':
                break

            generated_query = generate_mongodb_query(user_question, fields)
            if generated_query:
                print(f"Generated MongoDB Query: {generated_query}")
                with open(QUERIES_LOG_FILE, "a") as f:
                    f.write(f"User Query: {user_question}\n")
                    f.write(f"Query generated by Model - db.{COLLECTION_NAME}.find({generated_query})\n\n")

                output_choice = input("Choose output format ('display' or 'save'): ").lower()
                output_filename = None
                if output_choice == "save":
                    output_filename = input("Enter the filename for the saved CSV (e.g., output.csv): ")

                retrieve_and_present_data(MONGO_URI, DB_NAME, COLLECTION_NAME, generated_query, output_choice, output_filename)
            else:
                print("Failed to generate a query. Please try again.")
    else:
        print(f"The collection '{COLLECTION_NAME}' is empty.")

    client.close()

if __name__ == "__main__":
    main()


Successfully loaded 10 records from 'sample_data.csv' into 'products_data' in 'Automated_query'.

Enter your data query (or type 'exit' to quit): Find all products with a rating below 4.5 that have more than 200 reviews and are offered by the brand 'Nike' or 'Sony'.
Generated MongoDB Query: {
    "find": {
        "Rating": {"$lt", 4.5},
        "ReviewCount": {"$gt", 200},
        "$or": [
            {"Brand": "Nike"},
            {"Brand": "Sony"}
        ]
    }
}

Choose output format ('display' or 'save'): save
Enter the filename for the saved CSV (e.g., output.csv): query1.csv
Error: Generated MongoDB query is not valid JSON.

Enter your data query (or type 'exit' to quit): Which products in the Electronics category have a rating of 4.5 or higher and are in stock?
Generated MongoDB Query: python
query = {
    "$and": [
        {"Category": "Electronics"},
        {"$expr": {"$gt": [{"$avg": "$Rating"}, 4.5]}},  # filter products with rating >= 4.5
        {"Stock": True}  # filt

In [13]:
import csv
import pymongo
import pandas as pd
import json
import re
from langchain.llms import Ollama  # Make sure langchain and ollama are installed
from datetime import datetime

# --- Configuration ---
MONGO_URI = "mongodb://127.0.0.1:27017/"
DB_NAME = "Automated_query"
COLLECTION_NAME = "products_data"
OLLAMA_MODEL = "llama3"  # Replace with your preferred Ollama model
QUERIES_LOG_FILE = "Queries_generated.txt"

# --- Load CSV Data to MongoDB ---
def load_csv_to_mongodb(csv_file_path, mongo_host, database_name, collection_name):
    """Loads data from a CSV file into a MongoDB collection."""
    try:
        client = pymongo.MongoClient(mongo_host)
        db = client[database_name]
        collection = db[collection_name]
        collection.delete_many({})  # Clear existing data
        df = pd.read_csv(csv_file_path)
        data = df.to_dict(orient='records')
        if data:
            collection.insert_many(data)
            print(f"Successfully loaded {len(data)} records from '{csv_file_path}' "
                  f"into '{collection_name}' in '{database_name}'.")
        else:
            print(f"The CSV file '{csv_file_path}' is empty.")
    except Exception as e:
        print(f"Error loading CSV to MongoDB: {e}")
    finally:
        if 'client' in locals():
            client.close()

def fix_llm_query(raw_query):
    """
    Attempts to fix common LLM output issues and return a MongoDB filter dict.
    Accepts either a dict or a string.
    """
    if isinstance(raw_query, dict):
        # Remove unnecessary "find" key if present
        if "find" in raw_query and isinstance(raw_query["find"], dict):
            return raw_query["find"]
        return raw_query
    elif isinstance(raw_query, str):
        # Remove "find": { ... } wrapper if present
        match = re.match(r'^\s*["\']?find["\']?\s*:\s*({.*})\s*$', raw_query, re.DOTALL)
        if match:
            raw_query = match.group(1)
        # Replace invalid key,value no-colon pairs ({"$lt", 4.5}) with proper colon
        raw_query = re.sub(r'(\{)\s*["\']?(\$\w+)["\']?\s*,\s*([^\}]+)\}', r'\1"\2":\3}', raw_query)
        # Remove trailing commas in objects
        raw_query = re.sub(r',(\s*[}\]])', r'\1', raw_query)
        # Remove newlines for easier parsing
        raw_query = raw_query.replace('\n', ' ')
        # Try to parse as JSON
        try:
            return json.loads(raw_query)
        except Exception:
            pass
        # Try eval (dangerous, but as a last resort for trusted input)
        try:
            return eval(raw_query, {}, {})
        except Exception:
            pass
    return raw_query

# --- Generate MongoDB Query using LLM (Ollama) ---
def generate_mongodb_query(user_input, collection_fields):
    """Generates a MongoDB query string using Ollama."""
    try:
        llm = Ollama(base_url='http://localhost:11434', model=OLLAMA_MODEL)
        prompt_template = f"""
You are a helpful assistant that translates user queries into MongoDB find() filter objects (as Python dictionaries).
Do NOT include the 'find()' wrapper or any function call—just output the filter dictionary only.
The available fields in the MongoDB collection are: {', '.join(collection_fields)}.

User Query: {{user_query}}
MongoDB Query (as a Python dictionary, JSON-like, no function call, just the filter object):
"""
        prompt = prompt_template.format(user_query=user_input)
        raw_response = llm.invoke(prompt).strip()

        # Extract content between triple backticks or first JSON-like structure
        match = re.search(r"```(?:json)?\s*([\s\S]+?)```", raw_response)
        if match:
            cleaned = match.group(1)
        else:
            # Fallback: try to find the first { } block
            match = re.search(r"(\{[\s\S]+\})", raw_response)
            cleaned = match.group(1) if match else raw_response

        # Try to fix the query
        filter_query = fix_llm_query(cleaned)
        return filter_query
    except Exception as e:
        print(f"Error generating query with LLM: {e}")
        return None

# --- Retrieve and Present Data ---
def retrieve_and_present_data(mongo_host, database_name, collection_name, mongodb_query, output_format="display", output_filename=None):
    """Retrieves data from MongoDB using the generated query and presents it."""
    try:
        client = pymongo.MongoClient(mongo_host)
        db = client[database_name]
        collection = db[collection_name]

        # Ensure the query is a dict
        if isinstance(mongodb_query, str):
            query = fix_llm_query(mongodb_query)
        else:
            query = mongodb_query

        if not isinstance(query, dict):
            print("Error: Generated MongoDB query is not a valid dictionary.")
            return None

        results = list(collection.find(query))

        if results:
            print("\nRetrieved Data:")
            df = pd.DataFrame(results)
            if "_id" in df.columns:
                df = df.drop(columns=["_id"])

            if output_format == "display":
                print(df.to_string())
                return df
            elif output_format == "save" and output_filename:
                df.to_csv(output_filename, index=False)
                print(f"Data saved to '{output_filename}'")
                return df
            else:
                print("Invalid output format or filename provided.")
                return None
        else:
            print("No data found matching the query.")
            return pd.DataFrame()

    except Exception as e:
        print(f"Error retrieving data from MongoDB: {e}")
        return None
    finally:
        if 'client' in locals():
            client.close()

# --- Main Interaction Function ---
def main():
    csv_file = "sample_data.csv"
    load_csv_to_mongodb(csv_file, MONGO_URI, DB_NAME, COLLECTION_NAME)

    client = pymongo.MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    first_document = collection.find_one()

    if first_document:
        fields = list(first_document.keys())
        if "_id" in fields:
            fields.remove("_id")

        while True:
            user_question = input("\nEnter your data query (or type 'exit' to quit): ")
            if user_question.lower() == 'exit':
                break

            generated_query = generate_mongodb_query(user_question, fields)
            if generated_query:
                print(f"Generated MongoDB Query: {generated_query}")
                with open(QUERIES_LOG_FILE, "a") as f:
                    f.write(f"User Query: {user_question}\n")
                    f.write(f"Query generated by Model - db.{COLLECTION_NAME}.find({generated_query})\n\n")

                output_choice = input("Choose output format ('display' or 'save'): ").lower()
                output_filename = None
                if output_choice == "save":
                    output_filename = input("Enter the filename for the saved CSV (e.g., output.csv): ")

                retrieve_and_present_data(MONGO_URI, DB_NAME, COLLECTION_NAME, generated_query, output_choice, output_filename)
            else:
                print("Failed to generate a query. Please try again.")
    else:
        print(f"The collection '{COLLECTION_NAME}' is empty.")

    client.close()

if __name__ == "__main__":
    main()

Successfully loaded 10 records from 'sample_data.csv' into 'products_data' in 'Automated_query'.

Enter your data query (or type 'exit' to quit): Which products in the Electronics category have a rating of 4.5 or higher and are in stock?
Generated MongoDB Query: {'filter': {'$and': [{'Category': 'Electronics'}, {'Rating': {'$gte': 4.5}}, {'Stock': True}]}}
Choose output format ('display' or 'save'): save
Enter the filename for the saved CSV (e.g., output.csv): query1.csv
Error retrieving data from MongoDB: unknown operator: $and, full error: {'ok': 0.0, 'errmsg': 'unknown operator: $and', 'code': 2, 'codeName': 'BadValue'}

Enter your data query (or type 'exit' to quit): List products launched after January 1, 2022, in the Home & Kitchen or Sports categories with a discount of 10% or more, sorted by price in descending order.
Generated MongoDB Query: {     "launchDate": {"$gt": ISODate("2022-01-01T00:00:00.000Z")},     "$and": [         {"category": {"$in": ["Home & Kitchen", "Sports"]

In [22]:
import csv
import pymongo
import pandas as pd
import json
from langchain.llms import Ollama
from datetime import datetime

# --- Configuration ---
MONGO_URI = "mongodb://127.0.0.1:27017/"
DB_NAME = "Automated_query"
COLLECTION_NAME = "products_data"
OLLAMA_MODEL = "llama3"  # Replace with your preferred Ollama model
QUERIES_LOG_FILE = "Queries_generated.txt"

# --- Load CSV Data to MongoDB ---
def load_csv_to_mongodb(csv_file_path, mongo_host, database_name, collection_name):
    """Loads data from a CSV file into a MongoDB collection."""
    try:
        client = pymongo.MongoClient(mongo_host)
        db = client[database_name]
        collection = db[collection_name]
        collection.delete_many({})  # Clear existing data
        df = pd.read_csv(csv_file_path)
        data = df.to_dict(orient='records')
        if data:
            collection.insert_many(data)
            print(f"Successfully loaded {len(data)} records from '{csv_file_path}' "
                  f"into '{collection_name}' in '{database_name}'.")
        else:
            print(f"The CSV file '{csv_file_path}' is empty.")
    except Exception as e:
        print(f"Error loading CSV to MongoDB: {e}")
    finally:
        if 'client' in locals():
            client.close()

# --- Generate MongoDB Query using LLM (Ollama) ---
def generate_mongodb_query(user_input, collection_fields):
    """Generates a MongoDB query string using Ollama."""
    try:
        llm = Ollama(base_url='http://localhost:11434', model=OLLAMA_MODEL)
        prompt_template = f"""
        You are a helpful assistant that translates user queries into MongoDB find() queries.
        The available fields in the MongoDB collection are: {', '.join(collection_fields)}.

        User Query: {{user_query}}
        MongoDB Query (as a Python dictionary):
        """
        prompt = prompt_template.format(user_query=user_input)
        generated_query = llm(prompt).strip()
        return generated_query
    except Exception as e:
        print(f"Error generating query with LLM: {e}")
        return None

# --- Retrieve and Present Data ---
def retrieve_and_present_data(mongo_host, database_name, collection_name, mongodb_query_string, output_format="display", output_filename=None):
    """Retrieves data from MongoDB using the generated query and presents it."""
    try:
        client = pymongo.MongoClient(mongo_host)
        db = client[database_name]
        collection = db[collection_name]

        try:
            query = json.loads(mongodb_query_string)
        except json.JSONDecodeError:
            print("Error: Generated MongoDB query is not valid JSON.")
            return None

        results = list(collection.find(query))

        if results:
            print("\nRetrieved Data:")
            df = pd.DataFrame(results)
            if "_id" in df.columns:
                df = df.drop(columns=["_id"])

            if output_format == "display":
                print(df.to_string())
                return df
            elif output_format == "save" and output_filename:
                df.to_csv(output_filename, index=False)
                print(f"Data saved to '{output_filename}'")
                return df
            else:
                print("Invalid output format or filename provided.")
                return None
        else:
            print("No data found matching the query.")
            return pd.DataFrame()

    except Exception as e:
        print(f"Error retrieving data from MongoDB: {e}")
        return None
    finally:
        if 'client' in locals():
            client.close()

# --- Main Interaction Function ---
def main():
    csv_file = "sample_data.csv"
    load_csv_to_mongodb(csv_file, MONGO_URI, DB_NAME, COLLECTION_NAME)

    client = pymongo.MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    first_document = collection.find_one()

    if first_document:
        fields = list(first_document.keys())
        if "_id" in fields:
            fields.remove("_id")

        while True:
            user_question = input("\nEnter your data query (or type 'exit' to quit): ")
            if user_question.lower() == 'exit':
                break

            generated_query = generate_mongodb_query(user_question, fields)
            if generated_query:
                print(f"Generated MongoDB Query: {generated_query}")
                with open(QUERIES_LOG_FILE, "a") as f:
                    f.write(f"User Query: {user_question}\n")
                    f.write(f"Query generated by Model - db.{COLLECTION_NAME}.find({generated_query})\n\n")

                output_choice = input("Choose output format ('display' or 'save'): ").lower()
                output_filename = None
                if output_choice == "save":
                    output_filename = input("Enter the filename for the saved CSV (e.g., output.csv): ")

                retrieve_and_present_data(MONGO_URI, DB_NAME, COLLECTION_NAME, generated_query, output_choice, output_filename)
            else:
                print("Failed to generate a query. Please try again.")
    else:
        print(f"The collection '{COLLECTION_NAME}' is empty.")

    client.close()

if __name__ == "__main__":
    main()


Successfully loaded 10 records from 'sample_data.csv' into 'products_data' in 'Automated_query'.

Enter your data query (or type 'exit' to quit): Which products in the Electronics category have a rating of 4.5 or higher and are in stock?
Generated MongoDB Query: I'd be happy to help you with that!

Based on your query, I would translate it into the following MongoDB find() query:

```
{
    "find": {
        "Category": "Electronics",
        "$and": [
            {"Rating": {"$gte": 4.5}},
            {"Stock": True}
        ]
    }
}
```

In this query, we're using the `find()` method to retrieve documents from the collection that match the specified criteria.

We're filtering products by their category ("Electronics"), and then applying two conditions using `$and`:

1. The product's rating is 4.5 or higher (using `$gte`: greater than or equal to).
2. The product is in stock (`Stock: True`). Note that `True` represents a boolean value indicating whether the product is in stock or not

In [23]:
!pip install pymongo pandas langchain langchain-community


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import csv
import pymongo
import pandas as pd
import json
from langchain.llms import Ollama  # Ensure langchain and ollama are installed

# --- Configuration ---
MONGO_URI = "mongodb://127.0.0.1:27017/"  # Default MongoDB URI
DB_NAME = "Automated_query_AI"
COLLECTION_NAME = "data_collection"
OLLAMA_MODEL = "llama3"  # Replace with your preferred Ollama model
QUERIES_LOG_FILE = "Queries_generated.txt" # For logging generated queries

# --- 1. CSV Data Management ---
def load_csv_to_mongodb(csv_file_path, mongo_host, database_name, collection_name):
    """Loads data from a CSV file into a MongoDB collection."""
    try:
        client = pymongo.MongoClient(mongo_host)
        db = client[database_name]
        collection = db[collection_name]
        collection.delete_many({})  # Clear existing data for fresh loading
        df = pd.read_csv(csv_file_path)
        data = df.to_dict(orient='records')
        if data:
            collection.insert_many(data)
            print(f"Successfully loaded {len(data)} records from '{csv_file_path}' "
                  f"into '{collection_name}' in '{database_name}'.")
        else:
            print(f"The CSV file '{csv_file_path}' is empty.")
    except FileNotFoundError:
        print(f"Error: CSV file not found at '{csv_file_path}'.")
    except pymongo.errors.ConnectionFailure as e:
        print(f"Error: Could not connect to MongoDB: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during CSV loading: {e}")
    finally:
        if 'client' in locals():
            client.close()

# --- 2. Dynamic Query Generation using LLM ---
def generate_mongodb_query(user_input, collection_fields):
    """Generates a MongoDB query string based on user input and collection fields using Ollama."""
    try:
        llm = Ollama(base_url='http://localhost:11434', model=OLLAMA_MODEL)
        prompt_template = f"""
You are a helpful assistant that translates user questions into MongoDB find() queries.
The available fields in the MongoDB collection are: {', '.join(collection_fields)}.

User Question: {user_input}
MongoDB Query (as a JSON object):
"""
        prompt = prompt_template.format(user_input=user_input)
        generated_query = llm(prompt).strip()
        try:
            # Validate if the generated query is valid JSON
            json.loads(generated_query)
            return generated_query
        except json.JSONDecodeError:
            print(f"Warning: LLM generated an invalid JSON query: '{generated_query}'")
            return None
    except Exception as e:
        print(f"Error generating query with LLM: {e}")
        return None

# --- 3. Data Retrieval and Presentation ---
def retrieve_and_present_data(mongo_host, database_name, collection_name, mongodb_query_string, output_format="display", output_filename=None):
    """Retrieves data from MongoDB using the generated query and presents it."""
    try:
        client = pymongo.MongoClient(mongo_host)
        db = client[database_name]
        collection = db[collection_name]

        try:
            query = json.loads(mongodb_query_string)
        except json.JSONDecodeError:
            print("Error: Generated MongoDB query is not valid JSON.")
            return None

        results = list(collection.find(query))

        if results:
            print("\nRetrieved Data:")
            df = pd.DataFrame(results)
            if "_id" in df.columns:
                df = df.drop(columns=["_id"])

            if output_format == "display":
                print(df.to_string())
                return df
            elif output_format == "save" and output_filename:
                df.to_csv(output_filename, index=False)
                print(f"Data saved to '{output_filename}'")
                return df
            else:
                print("Invalid output format specified.")
                return None
        else:
            print("No data found matching the query.")
            return pd.DataFrame()

    except pymongo.errors.ConnectionFailure as e:
        print(f"Error: Could not connect to MongoDB: {e}")
    except Exception as e:
        print(f"An unexpected error occurred during data retrieval: {e}")
    finally:
        if 'client' in locals():
            client.close()

# --- 4. User Interaction ---
def main():
    csv_file = "sample_data.csv"  # Replace with the actual path to your CSV file
    load_csv_to_mongodb(csv_file, MONGO_URI, DB_NAME, COLLECTION_NAME)

    client = pymongo.MongoClient(MONGO_URI)
    db = client[DB_NAME]
    collection = db[COLLECTION_NAME]
    first_document = collection.find_one()

    if first_document:
        fields = list(first_document.keys())
        if "_id" in fields:
            fields.remove("_id")

        while True:
            user_question = input(f"\nEnter your question about the data (fields: {', '.join(fields)}) or type 'exit': ")
            if user_question.lower() == 'exit':
                break

            generated_query = generate_mongodb_query(user_question, fields)
            if generated_query:
                print(f"Generated MongoDB Query: {generated_query}")
                with open(QUERIES_LOG_FILE, "a") as f:
                    f.write(f"User Query: {user_question}\n")
                    f.write(f"Query generated by Model - db.{COLLECTION_NAME}.find({generated_query})\n\n")

                output_choice = input("Choose output format ('display' or 'save'): ").lower()
                output_filename = None
                if output_choice == "save":
                    output_filename = input("Enter the filename for the saved CSV (e.g., test_case1.csv): ")

                retrieve_and_present_data(MONGO_URI, DB_NAME, COLLECTION_NAME, generated_query, output_choice, output_filename)
            else:
                print("Failed to generate a valid query. Please try again.")
    else:
        print(f"The collection '{COLLECTION_NAME}' is empty. Please ensure the CSV was loaded correctly.")

    client.close()

if __name__ == "__main__":
    main()

Successfully loaded 10 records from 'sample_data.csv' into 'data_collection' in 'Automated_query_AI'.

Enter your question about the data (fields: ProductID, ProductName, Category, Price, Rating, ReviewCount, Stock, Discount, Brand, LaunchDate) or type 'exit': Find all products with a rating below 4.5 that have more than 200 reviews and are offered by the brand 'Nike' or 'Sony'.

The user question is: "Find all products with a rating below 4.5 that have more than 200 reviews and are offered by the brand 'Nike' or 'Sony'."

Here's the equivalent MongoDB find() query in JSON format:
```json
{
  "$and": [
    {
      "Rating": { "$lt": 4.5 }
    },
    {
      "ReviewCount": { "$gt": 200 }
    },
    {
      "$or": [
        { "Brand": "Nike" },
        { "Brand": "Sony" }
      ]
    }
  ]
}
```
In this query, we're using the `$and` operator to combine three conditions. The first condition is that the `Rating` field should be less than 4.5 (`"$lt": 4.5`). The second condition is that the